In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-training.csv', index_col=0)
df_test = pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-test.csv', index_col=0)

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train.info()

In [ ]:
df_test.info()

# Exploratory Data Analysis
### Missing Value

dari info data di atas diketahui untuk beberapa kolom memiliki missing value (null).
maka dari itu akan dilakukan proses pada missing value

In [ ]:
pd.DataFrame({'count': df_train.isnull().sum(), 'percentage': df_train.isnull().mean()*100})

In [ ]:
pd.DataFrame({'count': df_train.isnull().sum(), 'percentage': df_train.isnull().mean()*100})

* Untuk data train dan data test memiliki jumlah dan persentase null value yang sama yaitu kisaran 20% untuk Monthly Income dan 2,6% Number of dependents
* Untuk debt ratio dihitung berdasarkan monthly income, maka dapat dieskplor hubungan antara debt ratio dengan fitur yang mempunyai missing value

In [ ]:
df_train[df_train['MonthlyIncome'].isnull()][['NumberOfDependents', 'DebtRatio']].describe()

In [ ]:
df_train[df_train['NumberOfDependents'].isnull()][['MonthlyIncome', 'DebtRatio']].describe()

* dari tabel 2, dapat dilihat bahwa NumberOfDependents kosong, MonthlyIncome pun kosong sehingga dapat disimpulkan bahwa yang mengosongkan field NumberOfDependents juga mengosongkan field MonthlyIncome
* dari tabel 1, dapat dilihat bahwa yang MonthlyIncome-nya kosong, kebanyakan memiliki NumberOfDependents = 0. 
* Dari sini maka dapat disimpulkan, kita dapat melakukan imputation untuk NumberOfDependants. Dengan mengisi kolom yang kosong dengan angka 0

In [ ]:
df_train[df_train['MonthlyIncome'].notnull()][['DebtRatio']].describe()

* Dari tabel, dilihat bahwa yang mengisi MonthlyIncome memiliki DebtRatio yang rendah dibandingkan yang tidak mengisi
* Dapat dilihat juga bagi yang mengisi MonthlyIncome memiliki DebtRatio di bawah 1. Maka dapat kita anggap bagi yang tidak mengisi memiliki DebtRatio di atas 1

In [ ]:
[df_train[df_train['DebtRatio']>1]['MonthlyIncome'].isnull().sum()/len(df_train)*100, 
df_test[df_test['DebtRatio']>1]['MonthlyIncome'].isnull().sum()/len(df_test)*100]

In [ ]:
df_train[(df_train['DebtRatio']>1) & (df_train['MonthlyIncome'].notnull())][['MonthlyIncome']].describe()

* Tabel di atas berguna untuk menentukan value imputation bagi MonthlyIncome.
* Jika dilihat dari tabel sebelumnya untuk hasil rasio data train dan data set adalah ~18. Ini berarti bahwa sebagian besar data yang MonthlyIncome kosong memiliki DebtRatio di atas 1
* Maka dari itu untuk mendapatkan value imputation yang tepat, dapat dilakukan dengan mengambil median dari data dengan DebtRatio>1 dan MonthlyIncome yang tidak kosong 

Dari hasil, value imputation = 1577

In [ ]:
df_train['MonthlyIncome'].replace(np.nan, 1577, inplace = True)
df_train['NumberOfDependents'].replace(np.nan, 0, inplace = True)
df_test['MonthlyIncome'].replace(np.nan, 1577, inplace = True)
df_test['NumberOfDependents'].replace(np.nan, 0, inplace = True)

In [ ]:
df_train.describe()

* Dari info di atas, terdapat perbedaan jumlah yang besar antara 75% tile dan max value. 
* Dapat dilihat juga bahwa mean jauh lebih besar daripada median untuk kolom 'DebtRatio' 

Hal ini menyebabkan kemungkinan bahwa dataset ini memiliki extreme outlier.

Selanjutnya akan dilakukan eksplorasi data per kolom

### Revolving Utilization Of UnsecuredLines

In [ ]:
df_train['RevolvingUtilizationOfUnsecuredLines'].describe()

In [ ]:
df_train[df_train['RevolvingUtilizationOfUnsecuredLines'] > df_train['RevolvingUtilizationOfUnsecuredLines'].quantile(0.99)]['RevolvingUtilizationOfUnsecuredLines'].describe()

Jika dilihat dari info di atas, untuk 99% tile, perbedaan mean dengan median begitu besar.
Selanjutnya akan dilakukan plotting 

In [ ]:
fig, ax= figsize=(15,5)

sns.boxplot(x = np.array(df_train['RevolvingUtilizationOfUnsecuredLines']))

* Dari tabel, diketahui max 50708 dan 75% data pertama < 1. Sehingga dapat disimpulkan terdapat outlier pada fitur ini.
* Dalam keadaan normal, harusnya fitur RevolvingUtilizationOfUnsecuredLines memiliki nilai antara 0 dan 1. Tetapi dari dataset ini ada kemungkinan peminjam meminjam melebihi kredit limit
* Dari plot dapat dilihat distribusi untuk RevolvingUtilizationOfUnsecuredLines adalah <1 dan >10 

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,5))

sns.boxplot(x = df_train[df_train['RevolvingUtilizationOfUnsecuredLines'] < 1]['RevolvingUtilizationOfUnsecuredLines']
           , ax = ax[0])
ax[0].set_title('RevolvingUtilizationOfUnsecuredLines < 1')

sns.boxplot(x = df_train[df_train['RevolvingUtilizationOfUnsecuredLines'] > 10]['RevolvingUtilizationOfUnsecuredLines']
           , ax = ax[1])
ax[1].set_title('RevolvingUtilizationOfUnsecuredLines > 10')

In [ ]:
[df_train[df_train['RevolvingUtilizationOfUnsecuredLines'] <1]['RevolvingUtilizationOfUnsecuredLines'].count()/len(df_train) * 100,
df_train[df_train['RevolvingUtilizationOfUnsecuredLines'] >10]['RevolvingUtilizationOfUnsecuredLines'].count()/len(df_train) * 100]

* Dari plot dapat dilihat bahwa terdapat outlier yaitu yang berbentuk bukan box
* Untuk mengetahui proporsi antar outlier dan yang bukan, maka dilakukan perhitungan seperti code di atas. Dapat dilihat persentasenya adalah 97 untuk yang bukan dan 0,16 untuk outlier
* Karena proporsi outlier itu kecil dan membuat pengaruh besar pada mean maka outlier/fitur RevolvingUtilizationOfUnsecuredLines > 10 akan dihilangkan.

In [ ]:
df_train = df_train[df_train['RevolvingUtilizationOfUnsecuredLines']<=10]

### Debt Ratio

In [ ]:
df_train['DebtRatio'].describe()

* Sama seperti RevolvingUtilizationOfUnsecuredLines, DebtRatio juga memiliki outlier terlihat dari perbedaan yang begitu besar antar mean dan median serta 75%tile dengan max
* Dengan value normal, sama seperti RevolvingUtilizationOfUnsecuredLines, datanya terbagi menjadi nilai: <1, >10, dan 1<x<10

In [ ]:
fig,ax = figsize=(15,5)

sns.boxplot(x=df_train['DebtRatio'])

In [ ]:
fig,ax = plt.subplots(1, 3, figsize=(18,5))
sns.boxplot(x = df_train[df_train['DebtRatio'] < 1]['DebtRatio'], ax=ax[0])
sns.boxplot(x = df_train[df_train['DebtRatio'] > 10]['DebtRatio'], ax=ax[1])
sns.boxplot(x = df_train[(df_train['DebtRatio'] > 1) & (df_train['DebtRatio'] <= 10)]['DebtRatio'], ax=ax[2])

In [ ]:
pd.DataFrame({'below_1': df_train[df_train['DebtRatio'] < 1]['DebtRatio'].count()/len(df_train)*100,
             'beyond_10': df_train[df_train['DebtRatio'] >10]['DebtRatio'].count()/len(df_train)*100,
             'between_1-10': df_train[(df_train['DebtRatio'] > 1) & (df_train['DebtRatio'] <= 10)]['DebtRatio'].count()/len(df_train)*100
             }, index=[1])

* Dari data di atas dapat dilihat bahwa nilai > 10 menyumbang sekitar 20% dan ini merupakan outlier. Tetapi kita tidak dapat menghapusnya karena kita menggunakan DebtRatio ini sebagai patokan untuk mendapatkan value imputation MonthlyIncome

### Age

In [ ]:
df_train['age'].describe()

* Dari tabel di atas, tidak ada perbedaan yang begitu besar, sehingga kemungkinan adanya outlier itu kecil/tidak banyak

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,5))
sns.boxplot(x = df_train['age'], ax = ax[0])
ax[0].set_title('data training')
sns.boxplot(x = df_test['age'], ax = ax[1])
ax[1].set_title('data testing')

* Dari plot di atas terlihat ada yang janggal dimana ada 1 nilai bernilai 0. Mustahil seorang bayi dapat melakukan pinjaman
* Untuk nilai tersebut akan diganti dengan umur termuda yang terdapat di data yaitu 18

In [ ]:
df_train['age'].replace(0, 18, inplace = True)

### Number Past Due

In [ ]:
plt.figure(figsize=(10,5))

numberPastDue = df_train[['NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'NumberOfTime30-59DaysPastDueNotWorse']]

sns.boxplot(data = numberPastDue)

* Dari plot, dapat dilihat bahwa terdapat outlier dengan nilai ~90
* untuk melihat seberapa besar data outlier ini, maka akan dihitung persentasenya

In [ ]:
df_train[df_train['NumberOfTime30-59DaysPastDueNotWorse'] > 20].mean()*100

* Dari data di atas ternyata outlier pada data ini menyumbang distribusi yang cukup besar yaitu ~55% untuk fitur SeriousDlqin2yrs. Oleh karena ini, outlier ini tidak akan dihilangkan

### Number Of Open Credit Lines And Loans

In [ ]:
df_train['NumberOfOpenCreditLinesAndLoans'].describe()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18,5))
sns.histplot(x=df_train['NumberOfOpenCreditLinesAndLoans'], binwidth=2, ax=ax[0])
ax[0].set_title('data training')
sns.histplot(x=df_test['NumberOfOpenCreditLinesAndLoans'],binwidth=2, ax=ax[1])
ax[1].set_title('data testing')

* untuk fitur ini, dilihat dari plot, right-skewed. Tetapi karena nilainya tidak besar, jadi akan dibiarkan

### Number Real Estate Loans Or Lines

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18,5))
sns.histplot(x=df_train['NumberRealEstateLoansOrLines'], binwidth=1, ax=ax[0])
ax[0].set_title('data training')
sns.histplot(x=df_test['NumberRealEstateLoansOrLines'],binwidth=1, ax=ax[1])
ax[1].set_title('data testing')

* Sama seperti fitur yang sebelumnya, untuk ini pun dibiarkan, tidak perlu diberikan proses tambahan

### Number Of Dependents

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18,5))
sns.histplot(x=df_train['NumberOfDependents'], binwidth=1, ax=ax[0])
ax[0].set_title('data training')
sns.histplot(x=df_test['NumberOfDependents'],binwidth=1, ax=ax[1])
ax[1].set_title('data testing')

* Sama seperti sebelumnya, tak perlu diberi proses tambahan

# Modelling

### memilih model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc, f1_score, precision_score, classification_report, confusion_matrix
from sklearn.model_selection import KFold, cross_val_score

In [ ]:
from sklearn.model_selection import train_test_split
x = df_train.drop(['SeriousDlqin2yrs'], axis = 1)
y = df_train['SeriousDlqin2yrs']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

In [ ]:
kfold = KFold(n_splits=5)

clfs=[]
clfs.append(RandomForestClassifier())
clfs.append(LogisticRegression(solver='liblinear'))
clfs.append(XGBClassifier(eval_metric='auc'))

cv_res=[]
for clf in clfs:
    cv_res.append(cross_val_score(clf,x_train, y_train,scoring='roc_auc',cv=kfold))

cv_means, cv_std = [], []
for cv in cv_res:
    cv_means.append(cv.mean())
    cv_std.append(cv.std())
    
cv_res_df=pd.DataFrame({'cv_mean':cv_means,
                      'cv_std':cv_std,
                     'algorithm':['Random Forest','Logistic Regression','XGBoost']})


In [ ]:
cv_res_df

# Model XGBClassifier

* Dari tabel, ternyata hasil terbaik terdapat pada XGBClassifier. Maka dari itu akan digunakan XGBClassifier

In [ ]:
xgb = XGBClassifier(n_estimators = 300,random_state = 42)

xgb.fit(x_train, y_train)

#prediction
y_pred = xgb.predict(x_test)

In [ ]:
#predict probability
y_pred_proba = xgb.predict_proba(x_test)[:, 1]

In [ ]:
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize = (8,5))

sns.heatmap(cm, annot=True, fmt=".2f", linewidths=.5, square=True, cmap='BuPu_r');
plt.show()

In [ ]:
#classification report: f1-score, precision, recall, accuracy
print(classification_report(y_test, y_pred))

In [ ]:
#auc score
roc_auc_score(y_test, y_pred_proba)

In [ ]:
#roc curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.plot(fpr, tpr)
plt.plot(fpr, fpr, linestyle = '--', color = 'b')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC Curve')

# Result

In [ ]:
df_test_base = df_test.drop(['SeriousDlqin2yrs'], axis = 1)

xgb_clf_proba = xgb.predict_proba(df_test_base)[:, 1]

xgb_model = pd.DataFrame({'Id': df_test.index.values, 
                         'Probability': xgb_clf_proba})

xgb_model.to_csv("submission.csv", index=False)

xgb_model